In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys
import math
import os
import glob

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 23.4 ms, sys: 18 ms, total: 41.3 ms
Wall time: 338 ms


In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

In [4]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-all/Mask_ERF_iris-all.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [5]:
track = pd.read_csv('../saved-files/threshold-all/track-reset-all.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,84.600598,399.000000,4,10,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.144601e+06,316000.000000,70.490523,18.382036,1,0 days 00:00:00
1,0,2,101.255095,389.000000,5,10,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.161255e+06,306000.000000,70.353605,18.058626,2,0 days 00:00:00
2,1,2,104.000000,390.000000,1,10,154,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.164000e+06,307000.000000,70.327606,18.074493,2,0 days 00:05:00
3,0,4,112.472423,381.827530,14,10,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,3,0 days 00:00:00
4,1,4,113.727184,383.443874,16,10,155,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,3,0 days 00:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35549,300,71,281.407469,341.801921,7,15,35528,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.341407e+06,258801.920735,68.779128,16.307449,12343,0 days 00:00:00
35550,300,72,284.000000,349.225721,3,15,35529,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.344000e+06,266225.720790,68.748089,16.479746,12344,0 days 00:00:00
35551,300,89,213.928276,182.828784,14,20,35543,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.273928e+06,99828.783967,69.504222,12.513754,12345,0 days 00:00:00
35552,300,97,272.389563,284.000000,2,20,35551,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.332390e+06,201000.000000,68.912575,14.925450,12346,0 days 00:00:00


In [6]:
# # Changing time cell to timedelta
# time_cell = track['time_cell'].astype('timedelta64[ns]').values
# time_cell

### CALCULATING FOR DISTANCE

In [12]:
distance = []
for i in range(len(track)):
    lat_andenes = 69.141599 # latitude of Andenes
    #lat_andenes = 69.2412 # latiude of the radar
    lon_andenes = 15.684104 # latitude of Andenes
    # lon_andenes = 16.0030 # longitude of where the radar is located
    
# 69.2412° N, 16.0030° E
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [13]:
distance[:10]
# np.max(distance)

[182.4061549695696,
 162.99137822765474,
 160.98606513865352,
 149.72248345918496,
 149.3151648172964,
 149.2139703150614,
 148.58484315590556,
 147.91223348059512,
 141.3846314516514,
 139.95070287050123]

In [14]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 4.46 ms, sys: 88 µs, total: 4.55 ms
Wall time: 4.56 ms


In [15]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,1,84.600598,399.000000,4,10,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.144601e+06,316000.000000,70.490523,18.382036,1,0 days 00:00:00,182.406155
1,0,2,101.255095,389.000000,5,10,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.161255e+06,306000.000000,70.353605,18.058626,2,0 days 00:00:00,162.991378
2,1,2,104.000000,390.000000,1,10,154,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.164000e+06,307000.000000,70.327606,18.074493,2,0 days 00:05:00,160.986065
3,0,4,112.472423,381.827530,14,10,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.172472e+06,298827.530189,70.261556,17.831991,3,0 days 00:00:00,149.722483
4,1,4,113.727184,383.443874,16,10,155,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.173727e+06,300443.873660,70.248236,17.869335,3,0 days 00:05:00,149.315165


## _cells that lived for at least 10 mins old_ 

In [16]:
%%time
# cell_one_hour = np.timedelta64( 600, 's') #10 minutes in seconds
cell_times = np.timedelta64( 600, 's') #10 minutes in seconds

#convert time_cell to seconds
time_cell = track['time_cell'].astype('timedelta64[ns]').values

# track cell that live for at least 10 minutes long
# track_close_to_AMF1 = track[ time_cell >= cell_times ]
track_close_to_AMF1 = track[ (time_cell >= cell_times) & (track['distance'] <= 50) ]
# track_close_to_AMF1 = track[ ((track['distance'] <= 95) & (track['time_cell'].values >= cell_one_hour)) ]
track_close_to_AMF1= track_close_to_AMF1.drop_duplicates(subset=['cell']) # remove duplicates of cell_ids
track_close_to_AMF1.head()

CPU times: user 67.8 ms, sys: 0 ns, total: 67.8 ms
Wall time: 73.4 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
93,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,48.700314
175,2,202,289.009194,324.516441,4,20,433,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.349009e+06,241516.441368,68.726906,15.870312,118,0 days 00:10:00,46.759717
299,3,28,207.279024,309.481573,6,10,479,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.267279e+06,226481.573100,69.481954,15.704430,201,0 days 00:10:00,37.895779
316,3,135,280.007685,327.547475,4,15,537,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.340008e+06,244547.474570,68.805721,15.966160,212,0 days 00:10:00,39.049165
376,3,131,266.731982,325.601422,8,15,536,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.326732e+06,242601.421660,68.927977,15.952552,244,0 days 00:10:00,26.072877


In [17]:
# total track cells found.
len(track_close_to_AMF1) 

517

# tracking all cells that were at least 10 minutes old back in time to get all the cells

In [18]:
%%time
combined_tracks_AMF1 = []
for index, cell_id in enumerate(np.unique(track_close_to_AMF1['cell'].values)): # looped through tracks found with AMF1 site
    if cell_id in track['cell']: # checking if those cells(tracks) are within the original tracked dataframe
        track_i = track[track['cell'] == cell_id] # display it if it is within the tracked dataframe
        combined_tracks_AMF1.append(track_i) # append the results you get a new dataFrame calld combined_tracks_AMF1

# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
# tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1, keys=['df1', 'df2', 'df3', 'df4', 'df5', 'df6'])
tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1)
tracks_closer_AMF1

CPU times: user 239 ms, sys: 8.91 ms, total: 248 ms
Wall time: 254 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
90,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,56.002984
91,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,53.374413
92,2,25,194.671921,309.407829,10,10,325,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.254672e+06,226407.828582,69.596285,15.734259,58,0 days 00:10:00,50.652501
93,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,48.700314
94,4,31,201.000000,311.000000,1,10,630,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261000e+06,228000.000000,69.537483,15.758260,58,0 days 00:20:00,44.164815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35361,295,115,277.072836,343.163451,8,20,35258,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.337073e+06,260163.451363,68.817054,16.352027,12240,0 days 00:05:00,44.904769
35362,296,104,278.000000,343.813100,3,20,35315,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.338000e+06,260813.099642,68.807996,16.365254,12240,0 days 00:10:00,46.033241
35431,296,72,275.246502,319.000000,3,15,35289,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.335247e+06,236000.000000,68.856887,15.770719,12273,0 days 00:00:00,31.881102
35432,297,74,275.687095,321.000000,2,15,35345,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.335687e+06,238000.000000,68.851046,15.818199,12273,0 days 00:05:00,32.782985


In [19]:
# resetting the index
tracks_closer_AMF1_index_reset = tracks_closer_AMF1.reset_index(drop=True)
tracks_closer_AMF1_index_reset.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,56.002984
1,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,53.374413
2,2,25,194.671921,309.407829,10,10,325,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.254672e+06,226407.828582,69.596285,15.734259,58,0 days 00:10:00,50.652501
3,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,48.700314
4,4,31,201.000000,311.000000,1,10,630,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261000e+06,228000.000000,69.537483,15.758260,58,0 days 00:20:00,44.164815


In [20]:
# len(tracks_closer_AMF1_index_reset)

# Algorithm to calculate the corresponding reflectivity values (calculate for the max and mean dBZ) and the area

In [21]:
unique_cell_ids = track_close_to_AMF1['cell'].values # getting the cells_tracked
len(unique_cell_ids)

517

In [17]:
# len(np.unique(tracks_closer_AMF1_index_reset['cell']))
# Mask_ERF = iris.load('./saved-files/Mask_ERF_iris.nc')[0]
# Mask_ERF

In [22]:
%%time
# all masked cell in iris cube
masked_tracked_cell = []
for i, unique_value in enumerate(unique_cell_ids):
    # get masked cell based on cell id
    get_mask_cell = tobac.utils.mask_cell(mask=Mask_ERF, cell=unique_value, track=track, masked=False) 
    masked_tracked_cell.append(get_mask_cell)

CPU times: user 1.3 s, sys: 17.8 ms, total: 1.32 s
Wall time: 1.38 s


In [19]:
# masked_tracked_cell[:10]

In [25]:
%%time
# Convert the segmented/mask cells from iris cube to xrray
masked_values_to_xarray = []
for i, items in enumerate(masked_tracked_cell):
    mask_convert_xr = xr.DataArray.from_iris(masked_tracked_cell[i]) # convert iris cube to xarray data array
    masked_values_to_xarray.append(mask_convert_xr) # append the results to list
    # print(mask_convert_xr)

CPU times: user 1 s, sys: 9.79 ms, total: 1.01 s
Wall time: 1.04 s


In [26]:
%%time
# Getting the original masked reflectivity values
slice_cell_times = []
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id] 
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    # slice values based on time
    slice_cell_time = masked_values_to_xarray[i].sel(time=slice(first_time, last_time))
    slice_cell_times.append(slice_cell_time)
    # print(slice_cell_time)

CPU times: user 707 ms, sys: 5.87 ms, total: 713 ms
Wall time: 736 ms


In [23]:
# slice_cell_times[:5]

In [24]:
# type(slice_cell_times)

In [25]:
# %%time
# # Create an empty Dataset
# dataset = xr.Dataset()

# # Add each DataArray to the Dataset as a separate variable
# for index, data_array in enumerate(slice_cell_times):
#     var_name = f'data_array_{index}'  # Create a variable name for each DataArray
#     dataset[var_name] = data_array  # Add the DataArray to the Dataset

# # Save the Dataset as a NetCDF file
# dataset.to_netcdf('all_data_arrays.nc')

In [27]:
%%time
equivalent_reflectivity_factors = 10**((equivalent_reflectivity_factor/10))
# equivalent_reflectivity_factors = 10*np.log10(Z)

CPU times: user 3.93 s, sys: 256 ms, total: 4.19 s
Wall time: 4.48 s


## algorithm to extract the speed, area, maximum, minimum and mean reflectivity values based on cell_id and specific times

In [28]:
%%time
min_reflectivities = []; mean_reflectivities = []; max_reflectivities = []; areas = [];
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    #################################
    import warnings
    warnings.filterwarnings('ignore')
    #################################
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id]  # dataframe based on cell_id
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    
#     # # # # Calculating area
    # area = tobac.analysis.calculate_area(track, Mask_ERF)
#     # area = tobac.analysis.calculate_area(cell_ids, masked_tracked_cell[i], method_area=None)
#     area = np.absolute(area['area']/1000000) # area in kilometers
#     areas.append(area)
    
    # area1 = np.divide(area['area'],1000000)
    # area['area($km^2$)'] = area1


    # reflectivity_masked based on times
    reflectivity_masked = equivalent_reflectivity_factors.sel(time=slice(first_time, last_time))
    
    # LOOPING THROUGH THE AND GETTING THE VALUES
    for index in (range(len(reflectivity_masked))):
        mean_ref = reflectivity_masked[index].mean().item() # get mean values of reflectivity # in dBZ values
        # # Convert from dBZ to Z values
        # mean_ref_Z = 10**(mean_ref / 10)
        # # Convert it back to dBZ units
        # mean_ref_dBZ = 10*np.log10(mean_ref_Z)
        #####################################
        max_ref = reflectivity_masked[index].max().item() # get max values of reflectivity
        
        mean_reflectivities.append(mean_ref)
        max_reflectivities.append(max_ref)
        
# concatenate the area values to one series
# area1 = pd.concat(areas)

# calculate velocity
# velocity = tobac.analysis.calculate_velocity(tracks_closer_AMF1_index_reset)

CPU times: user 4.5 s, sys: 0 ns, total: 4.5 s
Wall time: 4.79 s


In [29]:
# converting the mean Z unit back to dBZ
mean_reflectivities1 = 10*np.log10(mean_reflectivities)

# converting the max Z unit back to dBZ
max_reflectivities1 = 10*np.log10(max_reflectivities)

In [30]:
np.min(mean_reflectivities1), np.max(mean_reflectivities1)

(12.70891348332567, 17.0430868015557)

In [31]:
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,56.002984
1,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,53.374413
2,2,25,194.671921,309.407829,10,10,325,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.254672e+06,226407.828582,69.596285,15.734259,58,0 days 00:10:00,50.652501
3,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,48.700314
4,4,31,201.000000,311.000000,1,10,630,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261000e+06,228000.000000,69.537483,15.758260,58,0 days 00:20:00,44.164815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,295,115,277.072836,343.163451,8,20,35258,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.337073e+06,260163.451363,68.817054,16.352027,12240,0 days 00:05:00,44.904769
2875,296,104,278.000000,343.813100,3,20,35315,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.338000e+06,260813.099642,68.807996,16.365254,12240,0 days 00:10:00,46.033241
2876,296,72,275.246502,319.000000,3,15,35289,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.335247e+06,236000.000000,68.856887,15.770719,12273,0 days 00:00:00,31.881102
2877,297,74,275.687095,321.000000,2,15,35345,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.335687e+06,238000.000000,68.851046,15.818199,12273,0 days 00:05:00,32.782985


In [32]:
# append max min and mean reflectivity values based on each cell id and times to the tracks_closer_AMF1_index_reset dataframe
# tracks_closer_AMF1_index_reset['speed'] = velocity
# tracks_closer_AMF1_index_reset['area1'] = area1
tracks_closer_AMF1_index_reset['mean dBZ'] = pd.Series(mean_reflectivities1)
tracks_closer_AMF1_index_reset['max dBZ'] = pd.Series(max_reflectivities1)

In [32]:
# tracks_closer_AMF1_index_reset.to_csv('../saved-files/threshold-all/final-tracks-relative-to-KAZR-threshold-all-excluding-area.csv', index=False)

In [34]:
area = tobac.calculate_area(tracks_closer_AMF1_index_reset, Mask_ERF, method_area=None)
# area1 = (c['area']/1000000)
area1 = np.divide(area['area'],1000000)
area['area($km^2$)'] = area1

In [35]:
final_tracks_thresh_all = tracks_closer_AMF1_index_reset.drop(columns=['area'])
final_tracks_thresh_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$)
0,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,56.002984,13.245867,34.567482,28.852936
1,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,53.374413,12.974534,35.577388,32.832786
2,2,25,194.671921,309.407829,10,10,325,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.254672e+06,226407.828582,69.596285,15.734259,58,0 days 00:10:00,50.652501,12.780750,34.063904,29.847634
3,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,48.700314,12.708913,36.584485,20.893174
4,4,31,201.000000,311.000000,1,10,630,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261000e+06,228000.000000,69.537483,15.758260,58,0 days 00:20:00,44.164815,12.845599,38.092506,18.904437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,295,115,277.072836,343.163451,8,20,35258,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.337073e+06,260163.451363,68.817054,16.352027,12240,0 days 00:05:00,44.904769,15.270663,40.109338,123.385353
2875,296,104,278.000000,343.813100,3,20,35315,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.338000e+06,260813.099642,68.807996,16.365254,12240,0 days 00:10:00,46.033241,15.390894,40.106802,167.168113
2876,296,72,275.246502,319.000000,3,15,35289,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.335247e+06,236000.000000,68.856887,15.770719,12273,0 days 00:00:00,31.881102,15.390894,40.106802,36.815098
2877,297,74,275.687095,321.000000,2,15,35345,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.335687e+06,238000.000000,68.851046,15.818199,12273,0 days 00:05:00,32.782985,15.206985,38.598690,47.761373


In [36]:
#reset cell_id starting from 1
final_tracks_thresh_all['cloud_id'] = final_tracks_thresh_all.groupby('cell').ngroup() + 1
final_tracks_thresh_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
0,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,56.002984,13.245867,34.567482,28.852936,1
1,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,53.374413,12.974534,35.577388,32.832786,1
2,2,25,194.671921,309.407829,10,10,325,2020-03-13 04:10:00,2020-03-13 04:10:00,-2.254672e+06,226407.828582,69.596285,15.734259,58,0 days 00:10:00,50.652501,12.780750,34.063904,29.847634,1
3,3,26,196.563912,310.050014,8,10,478,2020-03-13 04:15:00,2020-03-13 04:15:00,-2.256564e+06,227050.014423,69.578553,15.745626,58,0 days 00:15:00,48.700314,12.708913,36.584485,20.893174,1
4,4,31,201.000000,311.000000,1,10,630,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261000e+06,228000.000000,69.537483,15.758260,58,0 days 00:20:00,44.164815,12.845599,38.092506,18.904437,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874,295,115,277.072836,343.163451,8,20,35258,2020-03-14 04:35:00,2020-03-14 04:35:00,-2.337073e+06,260163.451363,68.817054,16.352027,12240,0 days 00:05:00,44.904769,15.270663,40.109338,123.385353,516
2875,296,104,278.000000,343.813100,3,20,35315,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.338000e+06,260813.099642,68.807996,16.365254,12240,0 days 00:10:00,46.033241,15.390894,40.106802,167.168113,516
2876,296,72,275.246502,319.000000,3,15,35289,2020-03-14 04:40:00,2020-03-14 04:40:00,-2.335247e+06,236000.000000,68.856887,15.770719,12273,0 days 00:00:00,31.881102,15.390894,40.106802,36.815098,517
2877,297,74,275.687095,321.000000,2,15,35345,2020-03-14 04:45:00,2020-03-14 04:45:00,-2.335687e+06,238000.000000,68.851046,15.818199,12273,0 days 00:05:00,32.782985,15.206985,38.598690,47.761373,517


In [37]:
final_tracks_thresh_all = final_tracks_thresh_all.sort_values(by=['timestr', 'time'])
final_tracks_thresh_all

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
0,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,56.002984,13.245867,34.567482,28.852936,1
5,0,204,283.801945,325.238055,5,20,118,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,242238.055457,68.773477,15.900718,118,0 days 00:00:00,41.882830,13.245867,34.567482,127.366079,2
1,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,53.374413,12.974534,35.577388,32.832786,1
6,1,206,286.616563,325.430992,6,20,274,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.346617e+06,242430.991644,68.747761,15.898351,118,0 days 00:05:00,44.670285,12.974534,35.577388,218.912429,2
8,1,123,199.469266,304.741309,3,15,218,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.259469e+06,221741.309190,69.557003,15.604983,201,0 days 00:00:00,46.345783,12.974534,35.577388,19.898611,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,298,71,253.293305,322.648456,20,15,35409,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.313293e+06,239648.456426,69.052631,15.914528,12209,0 days 00:30:00,13.484535,15.301029,39.603226,138.303634,512
2870,298,103,246.895198,358.913478,24,20,35435,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.306895e+06,275913.478290,69.073933,16.820395,12239,0 days 00:20:00,45.731168,15.301029,39.603226,325.364677,515
2878,298,32,275.000000,321.000000,1,10,35390,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.335000e+06,238000.000000,68.857285,15.819899,12273,0 days 00:10:00,32.109330,15.301029,39.603226,29.851796,517
2871,299,90,248.975468,361.633154,21,20,35489,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.308975e+06,278633.154472,69.052144,16.880836,12239,0 days 00:25:00,48.561337,15.240493,41.116443,390.037330,515


In [38]:
# all tracks
# final_tracks_thresh_10.to_csv('../cells-closer-to-AMF1/cells-closer-AMF1_allkm-KAZR-thres-all.csv', index=False)
final_tracks_thresh_all.to_csv('../saved-files/threshold-all/cells-allkm-KAZR-thres-all.csv', index=False)

In [39]:
tracks = pd.read_csv('../saved-files/threshold-all/cells-allkm-KAZR-thres-all.csv')
tracks.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ,area($km^2$),cloud_id
0,0,114,189.651343,306.234949,3,15,58,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.249651e+06,223234.949184,69.644657,15.666960,58,0 days 00:00:00,56.002984,13.245867,34.567482,28.852936,1
1,0,204,283.801945,325.238055,5,20,118,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.343802e+06,242238.055457,68.773477,15.900718,118,0 days 00:00:00,41.882830,13.245867,34.567482,127.366079,2
2,1,120,192.108135,307.717964,5,15,215,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.252108e+06,224717.963612,69.621054,15.698178,58,0 days 00:05:00,53.374413,12.974534,35.577388,32.832786,1
3,1,206,286.616563,325.430992,6,20,274,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.346617e+06,242430.991644,68.747761,15.898351,118,0 days 00:05:00,44.670285,12.974534,35.577388,218.912429,2
4,1,123,199.469266,304.741309,3,15,218,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.259469e+06,221741.309190,69.557003,15.604983,201,0 days 00:00:00,46.345783,12.974534,35.577388,19.898611,3


In [40]:
len(tracks)

2879

# Identifying Cells that were found in KAZR with C-band

### testing for cell all cells

In [43]:
def all_kazr_cells(start_time:str, end_time:str, CAO_date:str, folder_name:str, gif_name:str):
    """Generate a GIF animation of all cells within a given time range.

    Parameters:
        start_time (str): The start time of the animation in the format '"%H:%M:%S %d %b %Y".'.
        end_time (str): The end time of the animation in the format '"%H:%M:%S %d %b %Y".'.
        CAO_date(str): The times of CAO.
        folder_name (str): The name of the folder containing the cell data.
        name_of_gif (str): The desired name for the generated GIF file.

    Returns:
        animation of cell"""
    
        
    # read track files
    tracks = pd.read_csv('../saved-files/threshold-all/cells-allkm-KAZR-thres-all.csv')
    # converting time object to datetime format
    tracks['time'] = pd.to_datetime(tracks['time'])
    tracks['timestr'] = pd.to_datetime(tracks['timestr'])
    
    # mask the start and end time
    mask_date = (tracks['timestr'] >= start_time) & (tracks['timestr'] <= end_time)
    cell1 = tracks[(mask_date)].reset_index(drop=True)
    
    # get unique values
    idx = np.unique(cell1['cell'].values)
    
    # get all masked cell and convert to xarray
    convert_alls = []
    for ii, cell_id in enumerate(idx):
        # mask in iris cube
        mask_cell_all = tobac.utils.mask_cell(Mask_ERF, cell_id, tracks)
        # convert to xarray dataarray
        convert_all = xr.DataArray.from_iris(mask_cell_all)
        convert_alls.append(convert_all)
        
    
    # plots
    loop_over = cell1['frame'].values[:]
    cells = cell1['cell'].values[:]
    for ind, i in enumerate(loop_over):
        latA = 69.141281 #latitude of COMBLE site
        lonA = 15.684166-1 #longitude of COMBLE site -1
        xm, ym = 6,2.75 # extent from center point in lon and lat

        fig, ax = plt.subplots(1,1,figsize=(10,9),subplot_kw={'projection': ccrs.Orthographic(lonA,latA)})
        # xm, ym = 6,2.75 # extent from center point in lon and lat

        # extent = (lonA-xm, lonA+xm, latA-ym, latA+ym) # define extent map
        extent = (lonA-xm+3.3, lonA+xm-3.7, latA-ym+2, latA+ym-1.5) # define extent map
        ax.set_extent(extent) # set extent of map
        ax.coastlines(resolution='10m', color='black') # plot coastlines with high resolution: 10m
        ax.set_facecolor('white')
        ax.add_feature(cfeature.LAND, linewidth=1.5, alpha=0.5)

        # add grid lines
        gl = ax.gridlines(x_inline=False, alpha=0.3, color='white'); gl.bottom_labels=True; gl.left_labels=True
        gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER
        gl.xlabel_style = {'rotation': 45}; gl.ylabel_style = {'color': 'black'}


        #############################
        # loop through cell-id
        for cell_idx in range(0, len(idx)):
            # change time to moth, day, year, hours and minutes
            dt = masked_values_to_xarray[cell_idx]['time'].dt.strftime('%m-%d-%Y %H:%M').values  ### array 

             # fill in the place where there is cloud with 1 and place with no cloud to be zero
            cloud = np.asarray(convert_alls[cell_idx][i], dtype='float')
            invalid_cloud = np.greater(cloud, 0)
            cloud[invalid_cloud] = 1
            invalid_cloud1 = np.less(cloud, 1)
            cloud[invalid_cloud1] = np.nan


            cs=ax.pcolormesh(convert_alls[cell_idx]['lon'],
                        convert_alls[cell_idx]['lat'], 
                        # cloud*equivalent_reflectivity_factor.values[i],
                        cloud*equivalent_reflectivity_factor.values[i],
                        cmap='jet',
                        transform=ccrs.PlateCarree(),
                        vmin=-20,
                        vmax=40)


             # display each id on cell
            frame = cell1[cell1['frame']==i]
            longi = frame['longitude'].values[:]
            lati = frame['latitude'].values[:]
            areas = frame['area($km^2$)'].values[:]
            cell_id = frame['cell'].values[:]
            cloud_id = frame['cloud_id'].values[:]

            #     # check if area has value of nan
            if not math.isnan(frame['area($km^2$)'].values[0]):
                # for lo, la, ce, are in zip(all_long, all_lat, all_idx, all_area):
                for lon, lat, cell_ids, area in zip(longi, lati, cell_id, areas):
                    fontdict={'fontweight': 'extra bold'}
                    ax.text(lon, lat, cell_ids, transform=ccrs.PlateCarree(), fontsize=10, color='black', fontdict=fontdict)


        ax.plot(lonA+1,latA, color='red', marker='*', markersize = 7.5,transform=ccrs.PlateCarree()) # plot red star at location of Andenes
        ax.set_title(f"Nordic Radar Mosaic: {dt[i]} UTC", fontweight='bold', fontsize=16, pad=11)  # title of figure


        # Add a colorbar axis at the right of the graph
        cbar_ax = fig.add_axes([0.85, 0.11, 0.047, 0.77]) #lbwh



        cbar=fig.colorbar(cs, cax=cbar_ax, orientation='vertical')
        cbar.set_label("Reflectivity Factor [dBZ]", fontsize = 15)
        
        # Specify the directory and file name
        directory = f'./figures/{CAO_date}/{folder_name}/'
        file_name = f'{dt[i]}.png'
        
        # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"Directory '{directory}' created successfully.")

            

    
        # save figure
        # fig.savefig(os.path.join(directory, file_name), dpi=300)
        # plt.close()
        
        dirr = sorted(os.listdir(directory))
        if f'{dt[i]}.png' in dirr:
            # print(f'{dt[i]}.png already exist')
            pass
            plt.close()
        else:
            fig.savefig(os.path.join(directory, file_name), dpi=300)
            # print(f"Figures in {file_name} executed!")
            plt.close()
    print('\U0001f600\U0001f600\U0001f600\U0001f600EXECUTION DONE!!!\U0001f600\U0001f600\U0001f600\U0001f600')
    # gc.collect()

        
        
    # build gif
    
    imgs = sorted(glob.glob(f"{directory}/*png"))
    with imageio.get_writer(f"{directory}/{gif_name}.gif", mode='I', duration=0.7) as writer:
        for filename in imgs:
            image = imageio.v2.imread(filename)
            writer.append_data(image)
    print(f"Figures in {folder_name} executed!")

In [44]:
# # %%time
# one = all_kazr_cells(start_time='18:10:04 13 Mar 2020', end_time='18:28:04 13 Mar 2020', 
#                 CAO_date='Mar-13 & Mar-14',
#                 folder_name='cell1', 
#                 gif_name='cell-id1')
# one

In [45]:
# Data containing the ID, start date, and end date
kazr_cell_time = [
    (1, "08:12:28 13 Mar 2020", "09:06:36 13 Mar 2020"),
    (2, "09:12:48 13 Mar 2020", "09:58:16 13 Mar 2020"),
    (3, "10:00:12 13 Mar 2020", "10:11:56 13 Mar 2020"),
    (4, "10:14:32 13 Mar 2020", "10:29:00 13 Mar 2020"),
    (5, "10:22:40 13 Mar 2020", "10:46:56 13 Mar 2020"),
    (6, "10:42:32 13 Mar 2020", "10:59:04 13 Mar 2020"),
    (7, "11:00:32 13 Mar 2020", "11:32:28 13 Mar 2020"),
    (8, "11:30:44 13 Mar 2020", "11:51:52 13 Mar 2020"),
    (9, "11:48:08 13 Mar 2020", "12:09:04 13 Mar 2020"),
    (10, "12:03:08 13 Mar 2020", "12:38:04 13 Mar 2020"),
    (11, "12:32:48 13 Mar 2020", "13:14:32 13 Mar 2020"),
    (12, "13:53:00 13 Mar 2020", "14:05:16 13 Mar 2020"),
    (13, "14:03:28 13 Mar 2020", "14:31:20 13 Mar 2020"),
    (14, "15:09:36 13 Mar 2020", "15:45:20 13 Mar 2020"),
    (15, "15:48:00 13 Mar 2020", "16:12:44 13 Mar 2020"),
    (16, "16:12:00 13 Mar 2020", "16:33:44 13 Mar 2020"),
    (17, "16:29:40 13 Mar 2020", "16:38:20 13 Mar 2020"),
    (18, "16:44:24 13 Mar 2020", "17:13:32 13 Mar 2020"),
    (19, "17:12:56 13 Mar 2020", "17:45:48 13 Mar 2020"),
    (20, "18:10:04 13 Mar 2020", "18:28:04 13 Mar 2020"),
    (21, "18:27:28 13 Mar 2020", "18:50:04 13 Mar 2020"),
    (22, "19:15:32 13 Mar 2020", "19:54:00 13 Mar 2020"),
    (23, "19:47:12 13 Mar 2020", "20:15:52 13 Mar 2020"),
    (24, "20:14:12 13 Mar 2020", "20:48:12 13 Mar 2020"),
    (25, "20:48:36 13 Mar 2020", "21:21:40 13 Mar 2020"),
    (26, "21:27:40 13 Mar 2020", "21:50:48 13 Mar 2020"),
    (27, "21:48:20 13 Mar 2020", "22:13:00 13 Mar 2020"),
    (28, "22:28:24 13 Mar 2020", "23:00:00 13 Mar 2020"),
    (29, "23:38:56 13 Mar 2020", "00:16:28 14 Mar 2020"),
    (30, "00:13:24 14 Mar 2020", "00:30:44 14 Mar 2020"),
    (31, "01:06:40 14 Mar 2020", "01:24:08 14 Mar 2020"),
    (32, "02:19:20 14 Mar 2020", "02:34:36 14 Mar 2020"),
    (33, "02:43:24 14 Mar 2020", "03:12:44 14 Mar 2020"),
    (34, "03:17:16 14 Mar 2020", "03:41:20 14 Mar 2020"),
    (35, "03:57:16 14 Mar 2020", "04:31:56 14 Mar 2020"),
    (36, "04:27:40 14 Mar 2020", "04:50:20 14 Mar 2020"),
    (37, "04:49:56 14 Mar 2020", "05:19:48 14 Mar 2020")
]

In [46]:
for i in kazr_cell_time:
    # extract variables
    kazr_id, kazr_start_time, kazr_end_time = int(i[0]), i[1], i[2]
    ## display images and animations for all track-cells for March 13 and March 14
    all_kazr_cells(start_time=kazr_start_time, 
                   end_time=kazr_end_time,
                   CAO_date="Mar-13 & Mar-14",
                  folder_name=f'ID-{kazr_id}', 
                 gif_name=f'ID-{kazr_id}')

Directory './figures/Mar-13 & Mar-14/ID-1/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-1 executed!
Directory './figures/Mar-13 & Mar-14/ID-2/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-2 executed!
Directory './figures/Mar-13 & Mar-14/ID-3/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-3 executed!
Directory './figures/Mar-13 & Mar-14/ID-4/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-4 executed!
Directory './figures/Mar-13 & Mar-14/ID-5/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-5 executed!
Directory './figures/Mar-13 & Mar-14/ID-6/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-6 executed!
Directory './figures/Mar-13 & Mar-14/ID-7/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-7 executed!
Directory './figures/Mar-13 & Mar-14/ID-8/' created successfully.
😀😀😀😀EXECUTION DONE!!!😀😀😀😀
Figures in ID-8 executed!
Directory './figures/Mar-13 & Mar-14/ID-9/' created succ

# END 